In [1]:
redwine = sc.textFile('/wine/winequality-red.csv',3)

In [2]:
redwine.cache()

/wine/winequality-red.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [19]:
redwine_rdd = redwine.map(lambda x:x.split(';'))

In [20]:
redwine_table = redwine_rdd.collect()

In [21]:
headers = redwine_table.pop(0)
headers = [str(header.replace('"', '')) for header in headers]
headers

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [22]:
import pandas as pd
redwine_df = pd.DataFrame(redwine_table, columns=headers)

In [23]:
train = redwine_df.sample(frac=0.7)
test = redwine_df.loc[~redwine_df.index.isin(train.index)]

In [24]:
train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
533,10.3,0.27,0.24,2.1,0.072,15,33,0.9956,3.22,0.66,12.8,6
611,13.2,0.38,0.55,2.7,0.081,5,16,1.0006,2.98,0.54,9.4,5
580,12.3,0.5,0.49,2.2,0.089,5,14,1.0002,3.19,0.44,9.6,5
506,10.4,0.24,0.46,1.8,0.075,6,21,0.9976,3.25,1.02,10.8,7
352,7.7,0.665,0,2.4,0.09,8,19,0.9974,3.27,0.73,9.3,5


In [25]:
test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1,7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15,54,0.997,3.26,0.65,9.8,5
8,7.8,0.58,0.02,2,0.073,9,18,0.9968,3.36,0.57,9.5,7
13,7.8,0.61,0.29,1.6,0.114,9,29,0.9974,3.26,1.56,9.1,5
17,8.1,0.56,0.28,1.7,0.368,16,56,0.9968,3.11,1.28,9.3,5


In [26]:
print(train.shape,test.shape)

(1119, 12) (480, 12)


In [27]:
train_x=train.iloc[:,:-1]

In [28]:
train_y=train.iloc[:,-1]

In [29]:
test_x=train.iloc[:,:-1]
test_y=train.iloc[:,-1]

In [30]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,30))
mlp.fit(train_x, train_y)

/home/kbh/.local/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(hidden_layer_sizes=(50, 30))

In [31]:
pred = mlp.predict(test_x)

In [32]:
confusion_matrix = pd.crosstab(test_y,pred,rownames=['True'],
                              colnames=['Predicted'],
                              margins=True)

In [33]:
confusion_matrix

Predicted,4,5,6,7,All
True,,,,,
3,1,6,1,0,8
4,0,28,7,0,35
5,0,386,92,1,479
6,0,159,267,14,440
7,0,7,103,34,144
8,0,1,9,3,13
All,1,587,479,52,1119


In [34]:
print("test score : %s" %mlp.score(test_x,test_y))

test score : 0.613941018766756


In [35]:
print("train score : %s" %mlp.score(train_x,train_y))

train score : 0.613941018766756
